In [5]:
import numpy as np
import cupy as cp
import time

# Hàm cũ sử dụng NumPy
def weightss_to_lambda_numpy(weightss: np.ndarray, lambdas: np.ndarray) -> np.ndarray:
    num_qubits = weightss.shape[1]
    new_lambdas = np.zeros((4**num_qubits))
    for j, weights in enumerate(weightss):
        combinations = np.stack(np.meshgrid(*weights, indexing='ij'), axis=-1).reshape(-1, len(weights))
        new_lambdas += lambdas[j] * np.prod(combinations, axis=1)
    return new_lambdas

# Hàm mới sử dụng CuPy
def weightss_to_lambda_cupy(weightss: cp.ndarray, lambdas: cp.ndarray) -> cp.ndarray:
    num_terms, num_qubits, _ = weightss.shape
    new_lambdas = cp.zeros(4**num_qubits)
    for j in range(num_terms):
        weights = weightss[j]
        products = weights[0]
        for k in range(1, num_qubits):
            products = cp.outer(products, weights[k]).ravel()
        new_lambdas += lambdas[j] * products
    return new_lambdas

# Thiết lập tham số
num_qubits = 10  # 4^10 = 1,048,576 phần tử
num_terms = 200

# Tạo dữ liệu ngẫu nhiên cho NumPy
weightss_np = np.random.rand(num_terms, num_qubits, 4)
lambdas_np = np.random.rand(num_terms)

# Chuyển dữ liệu sang CuPy
weightss_cp = cp.asarray(weightss_np)
lambdas_cp = cp.asarray(lambdas_np)

# Benchmark hàm NumPy
start_time = time.time()
result_numpy = weightss_to_lambda_numpy(weightss_np, lambdas_np)
numpy_time = time.time() - start_time
print(f"Thời gian NumPy (CPU): {numpy_time:.4f} giây")

# Benchmark hàm CuPy
start_time = time.time()
result_cupy = weightss_to_lambda_cupy(weightss_cp, lambdas_cp)
cupy_time = time.time() - start_time
print(f"Thời gian CuPy (GPU): {cupy_time:.4f} giây")

if np.allclose(result_numpy, result_cupy):
    print("Kết quả giống nhau.")
else:
    print("Kết quả khác nhau.")

# Tính tốc độ tăng tốc
speedup = numpy_time / cupy_time
print(f"Tăng tốc: {speedup:.2f}x")

Thời gian NumPy (CPU): 19.0819 giây
Thời gian CuPy (GPU): 0.3014 giây
Kết quả giống nhau.
Tăng tốc: 63.32x


In [10]:
from gqimax.mapper import weightsss_to_lambdas
num_qubits = 3  # 4^10 = 1,048,576 phần tử
num_terms = 5
ks = [np.random.randint(1, num_terms) for _ in range(num_qubits)]

weightsss_np = [cp.random.rand(ks[i], num_qubits, 4) * (cp.random.rand(ks[i], num_qubits, 4) > 0.3) for i in range(num_qubits)]
lambdass_np = [cp.random.rand(ks[i]) for i in range(num_qubits)]
for i in range(num_qubits):
	print(weightsss_np[i].shape, lambdass_np[i].shape)
lambdass = weightsss_to_lambdas(weightsss_np, lambdass_np)

(3, 3, 4) (3,)
(4, 3, 4) (4,)
(3, 3, 4) (3,)


ValueError: operands could not be broadcast together with shapes (64,) (16,) (64,)

In [11]:
for i in range(num_qubits):
	print(f"lambdass[{i}]: {weightsss_np[i].shape}")

lambdass[0]: (3, 3, 4)
lambdass[1]: (4, 3, 4)
lambdass[2]: (3, 3, 4)


In [12]:
for i in range(num_qubits):
	print(f"lambdass[{i}]: {lambdass_np[i].shape}")

lambdass[0]: (3,)
lambdass[1]: (4,)
lambdass[2]: (3,)


In [13]:
for i in range(num_qubits):
    
    indices = cp.nonzero(lambdass[i])[0]
    print(indices)
    # lambdas = cp.array(lambdass[i])[indices]
    # print(lambdas.shape)

NameError: name 'lambdass' is not defined

In [ ]:
lambdass[0]

array([0.00716603, 0.        , 0.        , 0.00723714, 0.06154406,
       0.        , 0.01368813, 0.0195605 , 0.00911329, 0.        ,
       0.00118943, 0.00550251, 0.08158142, 0.        , 0.01923839,
       0.02252558, 0.03496448, 0.        , 0.        , 0.03531147,
       0.4022925 , 0.        , 0.09214885, 0.11953872, 0.05332943,
       0.        , 0.00800725, 0.02894194, 0.54142051, 0.        ,
       0.12951336, 0.14377764, 0.        , 0.        , 0.        ,
       0.        , 0.05900554, 0.        , 0.01467044, 0.01394007,
       0.00512727, 0.        , 0.00127478, 0.00121132, 0.08293111,
       0.        , 0.02061901, 0.01959248, 0.        , 0.        ,
       0.        , 0.        , 0.09377453, 0.        , 0.02331499,
       0.02215424, 0.00814851, 0.        , 0.00202595, 0.00192509,
       0.13179822, 0.        , 0.03276875, 0.03113734])

In [28]:
weightsss = cp.array([
    [[0,0, 1,4], [1,2,3,4]], 
    [[0,1,0,0], [1,2,0,0]], 
    [[1,0,0,4], [0,0, 1,4]]] )

In [37]:

from gqimax.mapper import weightss_to_lambda
from gqimax.utils import index_to_indices, index_to_word

lambdas = weightss_to_lambda(weightsss, lambdass_np[0])
lambdas

array([ 0.        ,  0.        ,  0.5873697 ,  2.34947881,  0.69422608,
        1.38845216,  0.        ,  0.        ,  0.58182811,  1.16365622,
        1.74548433,  2.32731244,  2.32731244,  4.65462487,  9.33141613,
       18.70716501])

In [62]:
indicess = []
for i in range(2):	
    indices = cp.nonzero(lambdas)[0]
    lambdas = lambdas[indices]
    new_indices = []
    for index in indices:
        new_indices.append(index_to_indices(int(index), 2))
    indicess.append(cp.array(new_indices))
print(indicess)

[array([[0, 0],
       [0, 1],
       [0, 2],
       [0, 3],
       [1, 0],
       [1, 1],
       [1, 2],
       [1, 3],
       [2, 0],
       [2, 1],
       [2, 2],
       [2, 3]]), array([[0, 0],
       [0, 1],
       [0, 2],
       [0, 3],
       [1, 0],
       [1, 1],
       [1, 2],
       [1, 3],
       [2, 0],
       [2, 1],
       [2, 2],
       [2, 3]])]
